# 01 — Tool Calling con LangChain

**Objetivo**: Implementar herramientas con `@tool`, conectarlas a un LLM via LangGraph, y usar structured output con Pydantic.

## Contenido
1. Herramientas con `@tool` decorator
2. Tool binding al modelo
3. Ejecucion manual de tools
4. LangGraph con `ToolNode`
5. Structured output con Pydantic

In [ ]:
import os
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

print("=" * 60)
print("TOOL CALLING CON LANGCHAIN")
print("=" * 60)

## 1. Herramientas con `@tool`

El decorator `@tool` convierte una funcion Python en una herramienta que el LLM puede invocar. LangChain genera automaticamente el schema JSON a partir del docstring y type hints.

In [ ]:
# ============================================================
# DEFINICION DE HERRAMIENTAS
# ============================================================

@tool
def buscar_comic(personaje: str, tema: str) -> str:
    """Busca informacion sobre un comic basado en personaje y tema.
    
    Args:
        personaje: Nombre del personaje (batman o spiderman).
        tema: Tema a buscar (origen, villanos, filosofia, etc).
    """
    import json as _json
    ruta = f"../data/{personaje.lower()}_comics.json"
    try:
        with open(ruta) as f:
            comics = _json.load(f)
        for comic in comics:
            if tema.lower() in comic.get("tema", "").lower() or tema.lower() in comic.get("arco", "").lower():
                return f"[{comic['titulo']}]: {comic['contenido'][:500]}..."
        return f"No se encontro informacion sobre {tema} para {personaje}."
    except FileNotFoundError:
        return f"No se encontro el archivo de datos para {personaje}."


@tool
def calcular_poder(personaje: str) -> str:
    """Calcula el nivel de poder de un personaje en escala 1-100.
    
    Args:
        personaje: Nombre del personaje.
    """
    poderes = {
        "batman": {"fuerza": 35, "inteligencia": 95, "tecnologia": 90, "combate": 85, "total": 76},
        "spiderman": {"fuerza": 70, "inteligencia": 85, "agilidad": 95, "sentido_aracnido": 90, "total": 85},
    }
    stats = poderes.get(personaje.lower())
    if stats:
        return json.dumps(stats, ensure_ascii=False)
    return f"No hay datos de poder para {personaje}."


@tool
def comparar_personajes(aspecto: str) -> str:
    """Compara Batman y Spider-Man en un aspecto especifico.
    
    Args:
        aspecto: Aspecto a comparar (fuerza, inteligencia, origen, filosofia, etc).
    """
    comparaciones = {
        "fuerza": "Spider-Man tiene fuerza proporcional de arana (10 ton). Batman tiene fuerza humana peak (~500 kg).",
        "inteligencia": "Ambos son genios. Batman: detective, estratega. Spider-Man: cientifico, inventor.",
        "origen": "Batman: trauma por asesinato de padres, elige la justicia. Spider-Man: mordedura de arana + culpa por muerte de tio Ben.",
        "filosofia": "Batman: no matar nunca, deontologia kantiana. Spider-Man: responsabilidad proporcional al poder, existencialismo.",
        "equipo": "Batman: Liga de la Justicia (estratega). Spider-Man: Avengers (corazon moral).",
    }
    resultado = comparaciones.get(aspecto.lower(), f"No hay comparacion predefinida para: {aspecto}")
    return resultado


tools = [buscar_comic, calcular_poder, comparar_personajes]

print("Herramientas definidas:")
for t in tools:
    print(f"  - {t.name}: {t.description[:80]}...")

## 2. Tool Binding

Conectamos las herramientas al modelo. El LLM ahora sabe que puede invocarlas.

In [ ]:
# ============================================================
# BINDING DE HERRAMIENTAS AL MODELO
# ============================================================

llm_with_tools = llm.bind_tools(tools)

# El modelo ahora puede decidir usar herramientas
response = llm_with_tools.invoke("Cual es el origen de Batman?")

print("Respuesta del modelo con tools bound:")
print(f"  Content: {response.content[:200] if response.content else '(vacio - uso tool call)'}")
print(f"  Tool calls: {len(response.tool_calls) if response.tool_calls else 0}")

if response.tool_calls:
    for tc in response.tool_calls:
        print(f"\n  Tool Call:")
        print(f"    Nombre: {tc['name']}")
        print(f"    Args: {tc['args']}")

## 3. Ejecucion Manual de Tools

Cuando el modelo decide usar una herramienta, debemos ejecutarla nosotros y devolver el resultado.

In [ ]:
# ============================================================
# EJECUCION MANUAL DEL TOOL CALL
# ============================================================

if response.tool_calls:
    # Buscar la herramienta por nombre
    tools_map = {t.name: t for t in tools}
    
    tool_messages = []
    for tc in response.tool_calls:
        tool_fn = tools_map[tc["name"]]
        resultado = tool_fn.invoke(tc["args"])
        tool_messages.append(ToolMessage(content=resultado, tool_call_id=tc["id"]))
        print(f"Ejecutado: {tc['name']}({tc['args']})")
        print(f"Resultado: {resultado[:200]}...\n")

    # Enviar el resultado al modelo para que genere la respuesta final
    messages = [HumanMessage("Cual es el origen de Batman?"), response] + tool_messages
    final_response = llm_with_tools.invoke(messages)
    print("Respuesta final:")
    print(final_response.content[:500])

## 4. LangGraph con ToolNode

`ToolNode` automatiza la ejecucion de herramientas dentro de un grafo LangGraph.

In [ ]:
# ============================================================
# LANGGRAPH CON TOOLNODE
# ============================================================

from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode


class AgentState(TypedDict):
    messages: Annotated[list, add_messages]


def call_model(state: AgentState) -> dict:
    """Nodo que invoca al LLM."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}


def should_continue(state: AgentState) -> str:
    """Decide si continuar con tools o terminar."""
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END


# Construir el grafo
graph = StateGraph(AgentState)
graph.add_node("agent", call_model)
graph.add_node("tools", ToolNode(tools))

graph.add_edge(START, "agent")
graph.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
graph.add_edge("tools", "agent")

app = graph.compile()

print("Grafo compilado exitosamente")
print("Nodos:", list(app.get_graph().nodes))

In [ ]:
# ============================================================
# EJECUCION DEL GRAFO
# ============================================================

result = app.invoke({"messages": [HumanMessage("Compara la fuerza de Batman y Spider-Man")]})

print("=" * 60)
print("EJECUCION DEL GRAFO COMPLETO")
print("=" * 60)

for msg in result["messages"]:
    tipo = msg.__class__.__name__
    if tipo == "HumanMessage":
        print(f"\n[USER] {msg.content}")
    elif tipo == "AIMessage":
        if msg.tool_calls:
            for tc in msg.tool_calls:
                print(f"\n[TOOL CALL] {tc['name']}({tc['args']})")
        if msg.content:
            print(f"\n[ASSISTANT] {msg.content[:400]}")
    elif tipo == "ToolMessage":
        print(f"\n[TOOL RESULT] {msg.content[:200]}...")

## 5. Structured Output con Pydantic

In [ ]:
# ============================================================
# STRUCTURED OUTPUT CON PYDANTIC
# ============================================================

class ComicAnalysis(BaseModel):
    """Analisis estructurado de un personaje de comic."""
    personaje: str = Field(description="Nombre del personaje")
    fortalezas: list[str] = Field(description="Lista de fortalezas principales")
    debilidades: list[str] = Field(description="Lista de debilidades")
    nivel_poder: int = Field(description="Nivel de poder 1-100", ge=1, le=100)
    resumen: str = Field(description="Resumen en una oracion")


structured_llm = llm.with_structured_output(ComicAnalysis)

analysis = structured_llm.invoke("Analiza a Spider-Man como personaje de comic")

print("=" * 60)
print("STRUCTURED OUTPUT")
print("=" * 60)
print(f"Personaje: {analysis.personaje}")
print(f"Fortalezas: {analysis.fortalezas}")
print(f"Debilidades: {analysis.debilidades}")
print(f"Nivel de poder: {analysis.nivel_poder}/100")
print(f"Resumen: {analysis.resumen}")
print(f"\nTipo: {type(analysis).__name__} (Pydantic validado)")

## Takeaways

1. `@tool` convierte funciones Python en herramientas invocables por el LLM
2. `bind_tools()` conecta herramientas al modelo sin ejecutarlas
3. La ejecucion de tools es **nuestra responsabilidad** (o de `ToolNode`)
4. `ToolNode` + `StateGraph` automatizan el loop ReAct
5. `with_structured_output()` garantiza respuestas tipadas con Pydantic